This project is to build a model for recongition of 3D printing mishaps and good prints

The following code has the following setup, it has a combiantion fo photos from "printing_errors_original" and "print_errors_2_modified_for_test"made into a folder called "images_combined

A training,validation and test set from "images_combined". This folder is organzied by class

The goal for this model is to train on a mix of lab setting and real world photos to see if the model can perform better than the model in "model_testing_seperated" jupyter notebook


In [2]:
#Import libraries that will be needed
import os
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
from torchvision import datasets
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torchvision.transforms import ToTensor
import random 
# Set random seed for reproducibility
manualSeed = 784
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
torch.use_deterministic_algorithms(True) # Needed for reproducible results


Random Seed:  784


In [16]:
#import folders for train and test
image_directory = "..\\images_combined"


class_names = sorted(os.listdir(image_directory))
class_mapping = {i: class_name for i, class_name in enumerate(class_names)}

print(class_mapping)

{0: 'good', 1: 'spaghetti', 2: 'stringing', 3: 'underextrusion'}


In [17]:
epochs = 100
lr = 0.000001
batch = 25
image_size = 227

In [18]:
# Set your image directory path
image_directory = "..\\images_combined"

# Create a custom dataset
custom_dataset = datasets.ImageFolder(root=image_directory,
                                      transform=transforms.Compose([
                                          transforms.Resize(image_size),
                                          transforms.CenterCrop(image_size),
                                          transforms.ToTensor(),
                                          transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                      ]))

In [19]:
###Modified Code, got rid of test set March 10, 2024
###

# Calculate the train, validation, and test sizes
train_size = int(0.7 * len(custom_dataset))  # 70% for training
val_size = int(0.1 * len(custom_dataset)) # 10% for validation
test_size = len(custom_dataset) - train_size - val_size  # Remaining 20% for testing

# Split the dataset into train, validation, and test sets
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    custom_dataset, [train_size, val_size, test_size])

# PyTorch Data Initialization
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch, shuffle=True, num_workers=2)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch, shuffle=False, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch, shuffle=False, num_workers=2)

# Using ngpu
ngpu = 1
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")




In [20]:
#Model Building

#PyTorch
#Second PyTorch Model
#AlexNet Architecture

class pyTorchModel_2(nn.Module):
    def __init__(self,ngpu, num_classes=4):
        super(pyTorchModel_2, self).__init__()
        self.ngpu = ngpu
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer5 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(9216, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes))
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [21]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

pymodel1 = pyTorchModel_2(ngpu).to(device)



Using cuda device


In [22]:
loss_fn = nn.CrossEntropyLoss()

In [23]:
#If only one model
optimizer1 = torch.optim.RMSprop(pymodel1.parameters(), lr=lr )

torch.use_deterministic_algorithms(False)


In [24]:
def train_loop(dataloader, model, loss_fn, optimizer, trainLoss):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    train_loss = 0
    num_batches = len(dataloader)

    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        #TrainLoss.append(loss.item())
        train_loss += loss.item()

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    train_loss /= num_batches
    trainLoss.append(train_loss)
    


def test_loop(dataloader, model, loss_fn, testLoss, acc):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            #testLoss.append(test_loss.item())
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    testLoss.append(test_loss)
    acc.append((100*correct))
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

def validate(model, dataloader, loss_fn, valLoss, acc):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    val_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            val_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    val_loss /= num_batches
    correct /= size
    valLoss.append(val_loss)
    acc.append((100 * correct))
    print(f"Validation Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {val_loss:>8f} \n")

In [25]:
trainLoss = []
testLoss = []
valLoss = []
acc = []

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, pymodel1, loss_fn, optimizer1, trainLoss)
    validate(pymodel1, val_dataloader, loss_fn, valLoss, acc)

print("Training and Validation Done!")

# After training, you can use the test_loop function to evaluate on a separate test set.
#test_loop(test_dataloader, pymodel1, loss_fn, testLoss, acc)

Epoch 1
-------------------------------
loss: 1.283471  [   25/11687]
loss: 0.921096  [ 2525/11687]
loss: 0.871764  [ 5025/11687]
loss: 1.169045  [ 7525/11687]
loss: 1.218938  [10025/11687]
Validation Error: 
 Accuracy: 63.7%, Avg loss: 0.837549 

Epoch 2
-------------------------------
loss: 0.896046  [   25/11687]
loss: 0.832135  [ 2525/11687]
loss: 0.850923  [ 5025/11687]
loss: 0.819049  [ 7525/11687]
loss: 0.546845  [10025/11687]
Validation Error: 
 Accuracy: 68.8%, Avg loss: 0.725164 

Epoch 3
-------------------------------
loss: 0.725020  [   25/11687]
loss: 0.818130  [ 2525/11687]
loss: 0.640722  [ 5025/11687]
loss: 0.590619  [ 7525/11687]
loss: 0.760907  [10025/11687]
Validation Error: 
 Accuracy: 76.8%, Avg loss: 0.591223 

Epoch 4
-------------------------------
loss: 0.709688  [   25/11687]
loss: 0.747252  [ 2525/11687]
loss: 0.793040  [ 5025/11687]
loss: 0.606168  [ 7525/11687]
loss: 0.634931  [10025/11687]
Validation Error: 
 Accuracy: 82.7%, Avg loss: 0.475360 

Epoch 5


In [26]:
def evaluate_predictions(model, dataloader):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            all_predictions.extend(pred.argmax(1).cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    return all_predictions, all_labels

In [27]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

# Assuming num_classes is the number of classes in your problem (4 in this case)
num_classes = 4

def calculate_multiclass_metrics(predictions, labels, num_classes):
    # Calculate confusion matrix
    conf_matrix = confusion_matrix(labels, predictions, labels=range(num_classes))

    # Calculate precision, recall, and F1 score for each class
    precision = precision_score(labels, predictions, average=None, labels=range(num_classes))
    recall = recall_score(labels, predictions, average=None, labels=range(num_classes))
    f1 = f1_score(labels, predictions, average=None, labels=range(num_classes))

    return conf_matrix, precision, recall, f1


###This is used to test predictions from the dataset that is SIMILAR to the training data
###
###
# Use the test_loop function to get predictions on the test set
test_predictions, test_labels = evaluate_predictions(pymodel1, test_dataloader)

# Calculate multiclass metrics
conf_matrix, precision, recall, f1 = calculate_multiclass_metrics(test_predictions, test_labels, num_classes)

# Print the results
print("Confusion Matrix:")
print(conf_matrix)
print("\nPrecision:")
print(precision)
print("\nRecall:")
print(recall)
print("\nF1 Score:")
print(f1)

Confusion Matrix:
[[1439    1    0    6]
 [   4   59    1    1]
 [  16    4  744    0]
 [  21    0    1 1043]]

Precision:
[0.9722973  0.921875   0.99731903 0.99333333]

Recall:
[0.99515906 0.90769231 0.97382199 0.97934272]

F1 Score:
[0.98359535 0.91472868 0.98543046 0.98628842]


In [14]:
from PIL import Image

def predict_single_image(model, image_path, class_labels):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')

    transform = transforms.Compose([
        transforms.Resize(image_size),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    input_image = transform(image).unsqueeze(0)  # Add batch dimension

    # Set the model to evaluation mode and make predictions
    model.eval()
    with torch.no_grad():
        input_image = input_image.to(device)
        output = model(input_image)

    # Interpret the model output
    predicted_index = output.argmax(1).item()
    predicted_class_name = class_mapping[predicted_index]
    


    return predicted_class_name, input_image


image_path = "test_image\\spag.jpg"
predicted_label, image_input = predict_single_image(pymodel1, image_path, class_mapping)
print(f"The predicted label for the image is: {predicted_label}")

FileNotFoundError: [Errno 2] No such file or directory: 'test_image\\spag.jpg'

In [28]:
model_path = 'CNNModelV0_3.pth'
torch.save(pymodel1.state_dict(), model_path)

In [50]:

#Convert model to a .onnx file so that I can convert it to something i can use
torch.onnx.export(pymodel1, image_input, 'CNN_Model.onnx', verbose=True)

In [4]:
import torchvision
from torch.utils.mobile_optimizer import optimize_for_mobile

pymodel1.eval()
example = torch.rand(1, 3, 227, 227)
traced_script_module = torch.jit.trace(pymodel1, example)
optimized_traced_model = optimize_for_mobile(traced_script_module)
optimized_traced_model._save_for_lite_interpreter("app/src/main/assets/model.ptl")

NameError: name 'pymodel1' is not defined